## Import data into pySpark
- https://github.com/titipata/pubmed_parser/wiki/Download-and-preprocess-MEDLINE-dataset

- MEDLINE BULK DOWNLOAD -
wget ftp://ftp.ncbi.nlm.nih.gov/pubmed/baseline/*.gz

- MEDLINE UPDATES -
wget ftp://ftp.ncbi.nlm.nih.gov/pubmed/updatefiles/*.gz

In [1]:
import os
from glob import glob
import pubmed_parser as pp
import findspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import Row

findspark.init()

In [2]:
conf = SparkConf()
spark = SparkSession.builder.\
    config(conf=conf).\
    getOrCreate()

### Process medline files
- https://github.com/titipata/pubmed_parser/wiki/Download-and-preprocess-MEDLINE-dataset

In [3]:
medline_files_rdd = spark.sparkContext.parallelize(glob('../data/pubmed/*.gz'), numSlices=1000)

In [4]:
parse_results_rdd = medline_files_rdd.\
    flatMap(lambda x: [Row(file_name=os.path.basename(x), **publication_dict) 
                       for publication_dict in pp.parse_medline_xml(x)])

In [14]:
medline_df = parse_results_rdd.toDF()

In [17]:
# medline_df.count()

In [6]:
# Process updates
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc

In [8]:
window = Window.partitionBy(['pmid']).orderBy(desc('file_name'))

In [12]:
windowed_df = medline_df.select(
    max('delete').over(window).alias('is_deleted'),
    rank().over(window).alias('pos'),
    '*')

In [13]:
windowed_df.count()

IllegalArgumentException: 'Unsupported class file major version 57'

In [11]:
windowed_df.where('is_deleted = False and pos = 1').\
    write.\
    parquet('medline_lastview.parquet', mode='overwrite')

Py4JJavaError: An error occurred while calling o72.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.

In [ ]:
# Calculate TF-IDF in all results

In [ ]:
sqlContext.registerDataFrameAsTable(df, "table1")